In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/AirQo_Low_Cost_Air_Quality_Monitor_Calibration/data/Train.csv" .
!cp "/content/drive/MyDrive/AirQo_Low_Cost_Air_Quality_Monitor_Calibration/data/Test.csv" .

In [3]:
!pip install catboost -q

     |████████████████████████████████| 69.2MB 54kB/s 


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [5]:
import warnings
warnings.simplefilter('ignore')

In [6]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# **Processing**

## checking hypothesis + feature engineering

In [7]:
data=pd.concat([train,test])

In [8]:
#basic time features
data['created_at']=pd.to_datetime(data['created_at'])
data['year']=data['created_at'].dt.year
data['month']=data['created_at'].dt.month
data['day']=data['created_at'].dt.day
data['hour']=data['created_at'].dt.hour
#data['dayofweek']=data['created_at'].dt.dayofweek
#data['dayofweek']=data['created_at'].dt.dayofweek
#data['is_weekend'] = (data['dayofweek'] >= 5)*1

In [9]:
near=data.copy()
near.sort_values(by=['created_at','pm2_5'],inplace=True)


near['closest_neighbour_target']=near['ref_pm2_5'].shift(1)
near['closest_neighbour_target'].fillna(method='ffill',inplace=True)
near['closest_neighbour_target_1']=near['ref_pm2_5'].shift(-1)
near['closest_neighbour_target_1'].fillna(method='bfill',inplace=True)

near['moy_neigh']=(near['closest_neighbour_target']+near['closest_neighbour_target_1'])/2

In [10]:
merge_cols=list(data.columns)
data=pd.merge(data,near,on=merge_cols,how="left")
del near

In [11]:
#baseline imputation 
data['temp'].fillna(data['temp'].mean(),inplace=True)
data['humidity'].fillna(data['humidity'].mean(),inplace=True)

In [12]:
#8.88(first)
data.drop('created_at',axis=1,inplace=True)
LE_cols = ['site']
for le_col in LE_cols :
    LE = LabelEncoder()
    data[le_col] = LE.fit_transform(data[le_col])

## time stamps features

In [13]:
# function to calculate number of days in a month /year 
import numpy
  
# explicit function to calculate
# number of days
def calcDays(month, year):
    date = 0
  
    # December case
    if month == 12:
        end = str(year)+'-'+str(month)+'-31'
        begin = str(year)+'-'+str(month)+'-01'
        date = 1
    else:
        if month == 9:
            endM = '-'+str(month+1)
            beginM = '-0'+str(month)
  
        # single digit months
        elif month < 9:
            endM = '-0'+str(month+1)
            beginM = '-0'+str(month)
  
        # double digit months
        else:
            endM = '-'+str(month+1)
            beginM = '-'+str(month)
  
        end = str(year)+endM+'-01'
        begin = str(year)+beginM+'-01'
  
    # return number of days
    return int((numpy.datetime64(end) - numpy.datetime64(begin)+date)/np.timedelta64(1,'D'))
    
  
  

def nb_days_year(year, month, day):
    s=0
    for i in range(1,month):
        s += calcDays(i,year)
    s += day
    return s
  



In [14]:
data['nb_days_in_year']=data.apply(lambda x : nb_days_year(x.year,x.month,x.day),axis=1)

In [15]:
def cum_day(year,nb_days_year):
    fst_year=2019
    year_days_count=0
    if year > 2019:
        end=str(year)+'-01-01'
        begin='2019-01-01'
        year_days_count=int((np.datetime64(end) - np.datetime64(begin))/np.timedelta64(1,'D'))
    return (year_days_count+nb_days_year)

data['date_days_since_2019']=data.apply(lambda x: cum_day(x.year,x.nb_days_in_year) , axis=1)

* lag features

In [16]:
#target mean encoding: since we have acess in test to past and future values , target mean without lag is not
#going to cause leakage 
cols=['site','date_days_since_2019']
data.set_index(cols,inplace=True)
data['lagged_target_by_days']=data.groupby(cols).ref_pm2_5.mean()
data['tard_std_by_day']=data.groupby(cols).ref_pm2_5.std()
#data['nb_measures']=data.groupby(cols).ref_pm2_5.count()/data.groupby(cols).size()
#std
#data['lagged_target_by_days_median']=data.groupby(cols).ref_pm2_5.median()
#data['lagged_target_by_days_sum']=data.groupby(cols).ref_pm2_5.sum()
#data['realtive_variation_per_day']=(data.groupby(cols).pm2_5.max()-data.groupby(cols).pm2_5.min())/(data.groupby(cols).ref_pm2_5.max()-data.groupby(cols).ref_pm2_5.min())

data.reset_index(inplace=True)


In [17]:
data['lagged_target_by_days'].fillna(method='ffill',inplace=True)
data['tard_std_by_day'].fillna(method='ffill',inplace=True)

In [18]:
data['month_count']=data['year'].apply(str)+data['month'].apply(str)
cols1=['site','month_count']
data.set_index(cols1,inplace=True)
data['target_monthly_mean']=data.groupby(cols1).ref_pm2_5.mean()
data.reset_index(inplace=True)
data.drop('month_count',axis=1,inplace=True)

we need to order the data so we can perform lags

In [19]:
cols=['site','date_days_since_2019']

odata=data.copy()
odata['src']=[0]*train.shape[0]+[1]*test.shape[0]
odata.sort_values(by=['date_days_since_2019','hour'],inplace=True)

In [20]:
#lagged target in the same hour from previous day/days (depends on disponibility in train)
odata['lag_by_hours']=odata.groupby(['site','hour']).ref_pm2_5.shift()
print(odata.lag_by_hours.isnull().sum())
odata['lag_by_hours']=odata.groupby(['site','hour']).lag_by_hours.ffill()
print(odata.lag_by_hours.isnull().sum())
odata.lag_by_hours.fillna(odata.lagged_target_by_days,inplace=True)
print(odata.lag_by_hours.isnull().sum())

2791
89
0


In [21]:
# target from the same hour in the next day 
odata['lag_by_hours_f']=odata.groupby(['site','hour']).ref_pm2_5.shift(-1)
print(odata.lag_by_hours_f.isnull().sum())
odata['lag_by_hours_f']=odata.groupby(['site','hour']).lag_by_hours_f.bfill()
print(odata.lag_by_hours_f.isnull().sum())
odata.lag_by_hours_f.fillna(odata.lagged_target_by_days,inplace=True)
print(odata.lag_by_hours_f.isnull().sum())

2792
91
0


In [22]:
#how much pm2-5 is close to the target + lag
odata['prox']=data['ref_pm2_5']-data['pm2_5']

In [23]:
#performing lag on prox
cols=['site']
odata['lagged_prox']=odata.groupby(cols).prox.shift()
print(odata.lagged_prox.isnull().sum())
odata['lagged_prox']=odata.groupby(cols).lagged_prox.ffill()
print(odata.lagged_prox.isnull().sum())
odata.lagged_prox.fillna(odata.lagged_target_by_days-odata.pm2_5,inplace=True)#mean by day
#odata.lagged_prox.fillna(odata.pm2_5,inplace=True)#mean by day

print(odata.lagged_prox.isnull().sum())


2735
3
0


In [24]:
#getting prox from next hours in a day
odata['lagged_prox_f']=odata.groupby(cols).prox.shift(-1)
print(odata.lagged_prox_f.isnull().sum())
odata['lagged_prox_f']=odata.groupby(cols).lagged_prox_f.bfill()
print(odata.lagged_prox_f.isnull().sum())
odata.lagged_prox_f.fillna(odata.lagged_target_by_days-odata.pm2_5,inplace=True)
#odata.lagged_prox_f.fillna(odata.pm2_5,inplace=True)
print(odata.lagged_prox_f.isnull().sum())
odata.drop('prox',axis=1,inplace=True)

2736
4
0


In [25]:
#summing with lag prox
odata['from_previous']=odata['pm2_5']+odata['lagged_prox']

#summing with lag prox f
odata['from_previous_f']=odata['pm2_5']+odata['lagged_prox_f']

#odata['from_previous_mean']=(odata['from_previous']+odata['from_previous_f'])

In [26]:
#same process with pm10
odata['prox10']=data['ref_pm2_5']-data['pm10']

In [27]:
odata['lagged_prox10']=odata.groupby(cols).prox10.shift()
print(odata.lagged_prox10.isnull().sum())
odata['lagged_prox10']=odata.groupby(cols).lagged_prox10.ffill()
print(odata.lagged_prox10.isnull().sum())
odata.lagged_prox10.fillna(odata.lagged_target_by_days-odata.pm10,inplace=True)
#odata.lagged_prox10.fillna(odata.pm10,inplace=True)
print(odata.lagged_prox10.isnull().sum())


2735
3
0


In [28]:
odata['lagged_prox10_f']=odata.groupby(cols).prox10.shift(-1)
print(odata.lagged_prox10_f.isnull().sum())
odata['lagged_prox10_f']=odata.groupby(cols).lagged_prox10_f.bfill()
print(odata.lagged_prox10_f.isnull().sum())
odata.lagged_prox10_f.fillna(odata.lagged_target_by_days-odata.pm10,inplace=True)
#odata.lagged_prox10_f.fillna(odata.pm10,inplace=True)
print(odata.lagged_prox10_f.isnull().sum())
odata.drop('prox10',axis=1,inplace=True)

2736
4
0


In [29]:
#summing with lag prox
odata['from_previous10']=odata['pm10']+odata['lagged_prox10']

#summing with lag prox f
odata['from_previous_f10']=odata['pm10']+odata['lagged_prox10_f']

#odata['from_previous10_mean']=(odata['from_previous10']+odata['from_previous_f10'])

In [30]:
#lagging target grouped by site and day
odata['lagged_target']=odata.groupby(cols).ref_pm2_5.shift()
print(odata.lagged_target.isnull().sum())
odata['lagged_target']=odata.groupby(cols).lagged_target.ffill()
print(odata.lagged_target.isnull().sum())
odata.lagged_target.fillna(odata.pm2_5,inplace=True)
print(odata.lagged_target.isnull().sum())


2735
3
0


In [31]:
#target from next hour in the same day
odata['lagged_target_reverse']=odata.groupby(cols).ref_pm2_5.shift(-1)
print(odata.lagged_target_reverse.isnull().sum())
odata['lagged_target_reverse']=odata.groupby(cols).lagged_target_reverse.bfill()
print(odata.lagged_target_reverse.isnull().sum())
odata.lagged_target_reverse.fillna(odata.pm2_5,inplace=True)
print(odata.lagged_target_reverse.isnull().sum())

2736
4
0


In [32]:
#lagging pm2_5 grouped only by site
odata['sensor_always_shift']=odata.groupby('site').pm2_5.shift()
odata.sensor_always_shift.fillna(odata.pm2_5,inplace=True)

shift temp and humidity

In [33]:
#merging new features with the original dataset
merge_cols=list(data.columns)
data=pd.merge(data,odata,on=merge_cols,how="left")

In [34]:
#mean of previous and next target value from the same day 
data['before_and_after']=(data['lagged_target']+data['lagged_target_reverse'])/2

In [35]:
from scipy.spatial.distance import cdist

def normalize(df,x):
    return (df[x]-df[x].min())/df[x].max()
     

def get_min_dist(df,cols):
    # compute distance matrix
    d=df.copy()
    for col in cols:
        
        d[col]=normalize(d,col)

    arr=d[cols].to_numpy()
    arr_train=d.loc[d.ID.isin(train.ID),cols].to_numpy()
    a = cdist(arr,arr_train)
    
    a[a==0]=a.max()
    # get index min of corresponding types
    idx=list(np.argmin(a,axis=1))
    print(len(idx))
    df['special_neigh']=df.reindex(index = idx)['ref_pm2_5'].values
    
    
    del d
    

get_min_dist(data,['pm10','pm2_5','date_days_since_2019'])

13665


basic features on sensor values

In [36]:
data['10:25_ratio']=data['pm2_5']/data['pm10']

In [37]:
data['comp_1']=data['pm10']-data['pm2_5']
data['comp_2']=data['s2_pm10']-data['s2_pm2_5']

In [38]:
data['moy_25']=(data['pm2_5']+data['s2_pm2_5'])/2
data['moy_10']=(data['pm10']+data['s2_pm10'])/2

In [39]:
data['moy_25_weighted']=(3*data['pm2_5']+data['s2_pm2_5'])/4
data['moy_10_weighted']=(3*data['pm10']+data['s2_pm10'])/4

In [40]:
data['mixed']=(data['pm2_5']+data['pm10'])/2
data['mixed_weighted']=(data['pm2_5']+3*data['pm10'])/4

In [41]:
#dropping constant features over site (for now i have just deleted lat and long)
del_cols=['lat','long','altitude', 'greenness', 'landform_90m',
       'landform_270m', 'population', 'dist_major_road']
data.drop(del_cols,axis=1,inplace=True)

# **Training By Site**

# **Modeling**

In [42]:
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
import random

In [43]:
from sklearn.model_selection import StratifiedKFold ,KFold
import lightgbm as lgb
import xgboost as xgb

In [44]:
# Install Catboost
import catboost as cat
from catboost import CatBoostRegressor, Pool

In [45]:
#preaparing data
ntrain=data[:train.shape[0]].drop(['src'],axis=1)
ntest=data[train.shape[0]:].drop(['src','ref_pm2_5'],axis=1)

In [46]:
train = ntrain.copy()
test = ntest.copy()

In [47]:
target = train['ref_pm2_5']

## 0. Utils

In [48]:
def Train_5Fold_lgbm(X,y,Test,site,kfold,params_lgb) :
  final_preds_per_location = []
  err_cb=[]
  site_oof_preds = np.zeros(len(X))
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]
      
      trn_data = lgb.Dataset(X_train, y_train)
      val_data = lgb.Dataset(X_test, y_test)

      model = lgb.train(params_lgb, trn_data, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 200)

      preds=model.predict(X_test, num_iteration=model.best_iteration)
      site_oof_preds[test_index] = preds
      err_cb.append(np.sqrt(mean_squared_error(y_test,preds)))
      
      test_pred = model.predict(Test.values, num_iteration=model.best_iteration)
      final_preds_per_location.append(test_pred)
  
  print(f"Site {site} --  Training RMSE :" ,np.mean(err_cb))
  return np.mean(final_preds_per_location,axis=0) , site_oof_preds

# ------------------------------------------------------------------------------------------------------------------------------

def Custom_train_Lgbm() :
  '''SEED The envierment'''
  seed = 0
  random.seed(seed)
  np.random.seed(seed)
  kfold=KFold(n_splits=5, random_state=1901,shuffle=True)
  
  final_preds=[]
  final_ids = []
  train_ids = []
  oof_prediction = []
  for site in tqdm_notebook([0,1,2], leave=False):
    print(60*'-')
    if site == 1 :
      params_lgb = {'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
              'learning_rate' : 0.06,'num_iterations': 3000,'max_depth' :7 ,'num_leaves' : 64,
              'feature_fraction': 0.6,'bagging_fraction': 1.0,'min_data_in_leaf':30,'reg_lambda' :75}
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',
                          'month_sin', 'month_cos','day_sin', 'day_cos', 'hour_sin', 'hour_cos']]
    elif site ==0 : 
      params_lgb = {'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
                    'learning_rate' : 0.06,'num_iterations': 2500,'max_depth' :7 ,'num_leaves' : 64,
                    'feature_fraction': 0.6,'bagging_fraction': 0.5,'min_data_in_leaf':15,'reg_lambda' :50,'verbose':-1}
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',]]
    else :
      params_lgb = {'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
                    'learning_rate' : 0.04,'num_iterations': 2500,'max_depth' :7 ,'num_leaves' : 64,
                    'feature_fraction': 0.6,'bagging_fraction': 0.5,'min_data_in_leaf':15,'reg_lambda' :50,'verbose':-1}
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',]]
    X = train[train['site']==site]  ; train_ids.extend(X['ID'].values)   ; y = X['ref_pm2_5']; X = X[features]
    Test = test[test['site']==site] ; final_ids.extend(Test['ID'].values); Test = Test[features]
    
    preds_per_location, site_oof_preds = Train_5Fold_lgbm(X=X,y=y,Test=Test,site=site,kfold=kfold,params_lgb=params_lgb)
    oof_prediction.extend(site_oof_preds) ; final_preds.extend(preds_per_location)
  
  oof_data = pd.DataFrame() ; oof_data['ID'] = train_ids ; oof_data['OOF'] = np.clip(oof_prediction,a_min=0,a_max=500) 
  preds_data = pd.DataFrame() ; preds_data['ID'] = final_ids ; preds_data['ref_pm2_5'] = np.clip(final_preds,a_min=0,a_max=500) 
  print()
  print(60*'#')
  return oof_data,preds_data

In [49]:
def Train_5Fold_xgb(X,y,Test,site,kfold,params_xgb) :
  final_preds_per_location = []
  err_cb=[]
  site_oof_preds = np.zeros(len(X))
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]
      
      trn_data = xgb.DMatrix(X_train, y_train)
      val_data = xgb.DMatrix(X_test, y_test)
      watchlist = [(trn_data, 'train'), (val_data, 'valid')]

      model = xgb.train(params_xgb, trn_data,10000, evals = watchlist, verbose_eval=0, early_stopping_rounds = 200)

      dX_test = xgb.DMatrix(X_test)
      preds = model.predict(dX_test)
      site_oof_preds[test_index] = preds
      err_cb.append(np.sqrt(mean_squared_error(y_test,preds)))
      
      dTest = xgb.DMatrix(Test.values)
      test_pred = model.predict(dTest)
      final_preds_per_location.append(test_pred)
  
  print(f"Site {site} --  Training RMSE :" ,np.mean(err_cb))
  return np.mean(final_preds_per_location,axis=0) , site_oof_preds

# ------------------------------------------------------------------------------------------------------------------------------

def Custom_train_xgb() :
  '''SEED The envierment'''
  seed = 0
  random.seed(seed)
  np.random.seed(seed)
  kfold=KFold(n_splits=5, random_state=1901,shuffle=True)
  
  final_preds=[]
  final_ids = []
  train_ids = []
  oof_prediction = []
  for site in tqdm_notebook([0,1,2], leave=False):
    print(60*'-')
    if site ==1 :
      params_xgb = {'objective': 'reg:squarederror','eval_metric': 'rmse','booster': 'gbtree', 
                    'n_estimators': 10000,  'max_depth': 7,'max_leaves': 80, 'learning_rate': 0.08, 'max_bin': 100,
                    'reg_alpha': 10, 'reg_lambda': 100,'subsample': 0.7 ,'colsample_bytree' : 0.6 }
      features = [x for x in train.columns 
              if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                          'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',]]
    elif site ==0 :
      params_xgb = {'objective': 'reg:squarederror','eval_metric': 'rmse','booster': 'gbtree', 
                    'n_estimators': 10000,  'max_depth': 7, 'learning_rate': 0.08, 'max_bin': 100,
                    'max_leaves': 64,'reg_alpha': 20, 'reg_lambda': 100,'subsample': 0.7 ,'colsample_bytree' : 0.6}
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',
                          'month_sin', 'month_cos','day_sin', 'day_cos', 'hour_sin', 'hour_cos']]
    else :
      params_xgb = {'objective': 'reg:squarederror','eval_metric': 'rmse','booster': 'gbtree', 
                    'n_estimators': 10000,  'max_depth': 7, 'learning_rate': 0.08, 'max_bin': 100,
                    'max_leaves': 64,'reg_alpha': 10, 'reg_lambda': 100,'subsample': 0.7 ,'colsample_bytree' : 0.6}
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',
                          'month_sin', 'month_cos','day_sin', 'day_cos', 'hour_sin', 'hour_cos']]
    X = train[train['site']==site]  ; train_ids.extend(X['ID'].values)   ; y = X['ref_pm2_5']; X = X[features]
    Test = test[test['site']==site] ; final_ids.extend(Test['ID'].values); Test = Test[features]
    
    preds_per_location, site_oof_preds = Train_5Fold_xgb(X=X,y=y,Test=Test,site=site,kfold=kfold,params_xgb=params_xgb)
    oof_prediction.extend(site_oof_preds) ; final_preds.extend(preds_per_location)
  
  oof_data = pd.DataFrame() ; oof_data['ID'] = train_ids ; oof_data['OOF'] = np.clip(oof_prediction,a_min=0,a_max=500) 
  preds_data = pd.DataFrame() ; preds_data['ID'] = final_ids ; preds_data['ref_pm2_5'] = np.clip(final_preds,a_min=0,a_max=500)
   
  print()
  print(60*'#')
  return oof_data,preds_data

In [50]:
def Train_5Fold_catboost_2(X,y,Test,site,kfold) :
  final_preds_per_location = []
  err_cb=[]
  site_oof_preds = np.zeros(len(X))
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]

      model = CatBoostRegressor(iterations=2000,learning_rate=0.03,depth=10,
                                loss_function='RMSE',random_seed=0,use_best_model=True,) 
      # CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',learning_rate=0.05, random_seed= 0,
      #                               use_best_model=True, ) 
      
      model.fit(X_train,y_train,eval_set=[(X_test, y_test)],early_stopping_rounds=20,verbose=0,)
      preds=model.predict(X_test)
      site_oof_preds[test_index] = preds
      err_cb.append(np.sqrt(mean_squared_error(y_test,preds)))
      
      test_pred = model.predict(Test.values)
      final_preds_per_location.append(test_pred)
  
  print(f"Site {site} --  Training RMSE :" ,np.mean(err_cb))
  return np.mean(final_preds_per_location,axis=0) , site_oof_preds

def Train_5Fold_catboost_2_site1(X,y,Test,site,kfold) :
  final_preds_per_location = []
  err_cb=[]
  site_oof_preds = np.zeros(len(X))
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]

      model = CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',learning_rate=0.05, random_seed= 0,
                                    use_best_model=True, ) 
      
      model.fit(X_train,y_train,eval_set=[(X_test, y_test)],early_stopping_rounds=20,verbose=0,)
      preds=model.predict(X_test)
      site_oof_preds[test_index] = preds
      err_cb.append(np.sqrt(mean_squared_error(y_test,preds)))
      
      test_pred = model.predict(Test.values)
      final_preds_per_location.append(test_pred)
  
  print(f"Site {site} --  Training RMSE :" ,np.mean(err_cb))
  return np.mean(final_preds_per_location,axis=0) , site_oof_preds
# ------------------------------------------------------------------------------------------------------------------------------ 

def Custom_train_catboost_2(features) :
  kfold=KFold(n_splits=5, random_state=1901,shuffle=True)
  
  final_preds=[]
  final_ids = []
  train_ids = []
  oof_prediction = []
  for site in tqdm_notebook([0,1,2], leave=False):
    if site ==1 :
      print(60*'-')
      X = train[train['site']==site]  ; train_ids.extend(X['ID'].values)   ; y = X['ref_pm2_5']; X = X[features]
      Test = test[test['site']==site] ; final_ids.extend(Test['ID'].values); Test = Test[features]
      
      preds_per_location, site_oof_preds = Train_5Fold_catboost_2_site1(X=X,y=y,Test=Test,site=site,kfold=kfold)
      oof_prediction.extend(site_oof_preds) ; final_preds.extend(preds_per_location)
    
    else :
      print(60*'-')
      X = train[train['site']==site]  ; train_ids.extend(X['ID'].values)   ; y = X['ref_pm2_5']; X = X[features]
      Test = test[test['site']==site] ; final_ids.extend(Test['ID'].values); Test = Test[features]
      
      preds_per_location, site_oof_preds = Train_5Fold_catboost_2(X=X,y=y,Test=Test,site=site,kfold=kfold)
      oof_prediction.extend(site_oof_preds) ; final_preds.extend(preds_per_location)
  
  oof_data = pd.DataFrame() ; oof_data['ID'] = train_ids ; oof_data['OOF'] = np.clip(oof_prediction,a_min=0,a_max=500) 
  preds_data = pd.DataFrame() ; preds_data['ID'] = final_ids ; preds_data['ref_pm2_5'] = np.clip(final_preds,a_min=0,a_max=500)
  
  print()
  print(60*'#')
  return oof_data,preds_data

In [51]:
def Train_5Fold_Stacking(X,y,Test,kfold) :
  final_preds = [] ; err_cb = []
  oof_stack = np.zeros(len(X)) ;
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]

      model = Ridge(alpha=0.01,random_state=42)
      model.fit(X_train,y_train)
      preds=model.predict(X_test)
      preds = np.clip(preds,a_min=0,a_max=500)
      oof_stack[test_index] = preds
      err_cb.append(mean_squared_error(y_test,preds,squared=False))
      
      test_pred = model.predict(Test.values)
      final_preds.append(test_pred)
  
  print(2*'--------------------------------------')
  print(f"Stacking RMSE :" ,np.mean(err_cb))
  return oof_stack,np.mean(final_preds,axis=0)

## 1. LGBM

In [52]:
oof_data_lgb,final_preds_lgb = Custom_train_Lgbm() 

------------------------------------------------------------
Site 0 --  Training RMSE : 7.87850548390176
------------------------------------------------------------
Site 1 --  Training RMSE : 19.231454686425717
------------------------------------------------------------
Site 2 --  Training RMSE : 9.355232910146768

############################################################


In [53]:
train_oof_data = pd.merge(train[['ID','ref_pm2_5']], oof_data_lgb,on='ID',how='left')
print('OOF RMSE : ',np.sqrt(mean_squared_error(train_oof_data['ref_pm2_5'],train_oof_data['OOF'])))

OOF RMSE :  11.399954952937337


## 2. xgb

In [54]:
oof_data_xgb,final_preds_xgb = Custom_train_xgb() 

------------------------------------------------------------
Site 0 --  Training RMSE : 8.019688017055989
------------------------------------------------------------
Site 1 --  Training RMSE : 19.396098267685183
------------------------------------------------------------
Site 2 --  Training RMSE : 9.356546743945694

############################################################


In [55]:
train_oof_data = pd.merge(train[['ID','ref_pm2_5']], oof_data_xgb,on='ID',how='left')
print('OOF RMSE : ',np.sqrt(mean_squared_error(train_oof_data['ref_pm2_5'],train_oof_data['OOF'])))

OOF RMSE :  11.480550679012481


## 4. CATBOOST_2

In [56]:
def Train_5Fold_catboost_2(X,y,Test,site,kfold,cat_params) :
  final_preds_per_location = []
  err_cb=[]
  site_oof_preds = np.zeros(len(X))
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]

      model = CatBoostRegressor(**cat_params) 
      
      model.fit(X_train,y_train,eval_set=[(X_test, y_test)],early_stopping_rounds=20,verbose=0,)
      preds=model.predict(X_test)
      site_oof_preds[test_index] = preds
      err_cb.append(np.sqrt(mean_squared_error(y_test,preds)))
      
      test_pred = model.predict(Test.values)
      final_preds_per_location.append(test_pred)
  
  print(f"Site {site} --  Training RMSE :" ,np.mean(err_cb))
  return np.mean(final_preds_per_location,axis=0) , site_oof_preds

# ------------------------------------------------------------------------------------------------------------------------------ 

def Custom_train_catboost_2() :
  kfold=KFold(n_splits=5, random_state=1901,shuffle=True)
  
  final_preds=[]
  final_ids = []
  train_ids = []
  oof_prediction = []
  for site in tqdm_notebook([0,1,2], leave=False):
    print(60*'-')
    if site ==1 :
      cat_params = {'eval_metric': 'RMSE','learning_rate': 0.05,'n_estimators': 5000,'random_seed': 0,'use_best_model': True}   
      
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',]]
    else :
      cat_params = {'depth': 8,'iterations': 2000,'learning_rate': 0.03,'loss_function': 'RMSE','random_seed': 0,'use_best_model': True}
      
      features = [x for x in train.columns 
            if x not in ['ID', 'created_at','dist_major_road' , 'greenness' , 'population','lat' , 'altitude','landform_90m','landform_270m',
                         'ref_pm2_5','long','pm10_div_s2_pm10','pm2_5_div_s2_pm2_5','pm10_div_s2_pm2_5','pm2_5_div_s2_pm10','month_day',
                          'month_sin', 'month_cos','day_sin', 'day_cos', 'hour_sin', 'hour_cos']]
    X = train[train['site']==site]  ; train_ids.extend(X['ID'].values)   ; y = X['ref_pm2_5']; X = X[features]
    Test = test[test['site']==site] ; final_ids.extend(Test['ID'].values); Test = Test[features]
    
    preds_per_location, site_oof_preds = Train_5Fold_catboost_2(X=X,y=y,Test=Test,site=site,kfold=kfold,cat_params=cat_params)
    oof_prediction.extend(site_oof_preds) ; final_preds.extend(preds_per_location)
  
  oof_data = pd.DataFrame() ; oof_data['ID'] = train_ids ; oof_data['OOF'] = np.clip(oof_prediction,a_min=0,a_max=500) 
  preds_data = pd.DataFrame() ; preds_data['ID'] = final_ids ; preds_data['ref_pm2_5'] = np.clip(final_preds,a_min=0,a_max=500)
  
  print()
  print(60*'#')
  return oof_data,preds_data

In [57]:
oof_data_catboost_2,final_preds_catboost_2 = Custom_train_catboost_2()

------------------------------------------------------------
Site 0 --  Training RMSE : 7.695895359832138
------------------------------------------------------------
Site 1 --  Training RMSE : 19.181995622361768
------------------------------------------------------------
Site 2 --  Training RMSE : 9.18989447989124

############################################################


In [58]:
train_oof_data = pd.merge(train[['ID','ref_pm2_5']], oof_data_catboost_2,on='ID',how='left')
print('OOF RMSE : ',np.sqrt(mean_squared_error(train_oof_data['ref_pm2_5'],train_oof_data['OOF'])))

OOF RMSE :  11.301870102312222


## 5. Stacking

In [59]:
from sklearn.linear_model import Ridge ,LinearRegression

In [60]:
stacking_train = oof_data_catboost_2.copy() ; del stacking_train['OOF'] ;
stacking_train['Hela_App1_preds_cat'] =  oof_data_catboost_2['OOF'] ; 
stacking_train['Hela_App1_preds_xgb'] =  oof_data_xgb['OOF'] ;
stacking_train['Hela_App1_preds_lgb'] =  oof_data_lgb['OOF']
stacking_train = pd.merge(train[['ID','ref_pm2_5']], stacking_train,on='ID',how='left')
stacking_train.rename({'ref_pm2_5': 'target'})

stacking_test = final_preds_catboost_2.copy() ; del stacking_test['ref_pm2_5']
stacking_test['Hela_App1_preds_cat'] =  final_preds_catboost_2['ref_pm2_5'] ; 
stacking_test['Hela_App1_preds_xgb'] =  final_preds_xgb['ref_pm2_5'] ;
stacking_test['Hela_App1_preds_lgb'] =  final_preds_lgb['ref_pm2_5']

In [61]:
cols = ['Hela_App1_preds_cat', 'Hela_App1_preds_xgb', 'Hela_App1_preds_lgb']

X , y , Test = stacking_train[cols] , stacking_train['ref_pm2_5'] , stacking_test[cols]
KFOLD = KFold(n_splits=5,random_state=1901,shuffle=True)

oof_stack,stack_preds = Train_5Fold_Stacking(X=X,y=y,Test=Test,kfold=KFOLD)

----------------------------------------------------------------------------
Stacking RMSE : 11.007931207317942


In [65]:
print('Stacking OOF',mean_squared_error(y,oof_stack,squared=False))

Stacking OOF 11.200203103871619


In [63]:
stacking_train.to_csv('Hela_App1_stacking_train.csv',index=False)
stacking_test.to_csv('Hela_App1_stacking_test.csv',index=False)